In [ ]:
using Pkg;Pkg.activate(localprojectdir())
using PyPlot, Plots

In [ ]:
using FrameFunWavelets, PyPlot, LinearAlgebra, WaveletsEvaluation

# DWT and iDWT

In [ ]:
dict = DaubechiesWaveletBasis(3,9)
DWT = Matrix(DiscreteWaveletTransform(dict))
iDWT = Matrix(InverseDiscreteWaveletTransform(dict));

In [ ]:
P = imshow(DWT,cmap="gray_r",norm=matplotlib.colors.LogNorm(eps()))
P.figaspect=1
PyPlot.savefig("DWT")

In [ ]:
P = imshow(iDWT,cmap="gray_r",norm=matplotlib.colors.LogNorm(eps()))
P.figaspect=1
colorbar()
PyPlot.savefig("iDWT")

In [ ]:
for p in 1:6
    iseven(p) ? (qs = 2:2:6) : (qs = 1:2:6)
    for q in qs
        @show p,q, opnorm(Matrix(DiscreteWaveletTransform(CDFWaveletBasis(p,q,5))))
    end
    
end
    

In [ ]:
for p in 1:6
    iseven(p) ? (qs = 2:2:6) : (qs = 1:2:6)
    for q in qs
        @show p,q, opnorm(Matrix(InverseDiscreteWaveletTransform(CDFWaveletBasis(p,q,5,DWT.Prl,Float64,false))))
    end
    
end

# Wavelets

In [ ]:
using WaveletsEvaluation, PGFPlotsX, DocumentPGFPlots, Glob

In [ ]:
# rm *.{fls,log,fdb*,aux,stderr,stdout}
function removefiles() 
    rm.(glob("*.fls"))
    rm.(glob("*.log"))
    rm.(glob("*.fdb*"))
    rm.(glob("*.aux"))
    rm.(glob("*.stderr"))
    rm.(glob("*.stdout")) 
end

function plotwavelet(w::DaubechiesWavelet, d=10)
    y1, x1 = evaluate_in_dyadic_points(Primal, scaling, w, 0, 0, d;points=true)
    y2, x2 = evaluate_in_dyadic_points(Primal, wavelet, w, 0, 0, d;points=true)

    P = @pgf Axis({width=".5\\textwidth",height=".25\\textwidth",},
        PlotInc({mark="none"},Table(x1,y1)),
        PlotInc({mark="none"},Table(x2,y2))
        )
    DocumentPGFPlots.savefigs(WaveletsEvaluation.DWT.name(w), P)
    removefiles()
    P
end
function plotwavelet(w::CDFWavelet, d=10)
    y1, x1 = evaluate_in_dyadic_points(Primal, scaling, w, 0,0,d;points=true)
    y2, x2 = evaluate_in_dyadic_points(Primal, wavelet, w, 0,0,d;points=true)

    y3, x3 = evaluate_in_dyadic_points(Dual, scaling, w, 0,0,d;points=true)
    y4, x4 = evaluate_in_dyadic_points(Dual, wavelet, w, 0,0,d;points=true)

    
    P1 = @pgf Axis({},
        PlotInc({mark="none"},Table(x1,y1)),
        PlotInc({mark="none"},Table(x2,y2))
        )
    P2 = @pgf Axis({},
        PlotInc({mark="none"},Table(x3,y3)),
        PlotInc({mark="none"},Table(x4,y4))
        )
    DocumentPGFPlots.savefigs(WaveletsEvaluation.DWT.name(w)*"primal", P2)
    DocumentPGFPlots.savefigs(WaveletsEvaluation.DWT.name(w)*"dual", P2)
    P = @pgf GroupPlot({width=".5\\textwidth",height=".25\\textwidth",group_style = {group_size="1 by 2",},},
            {},
            PlotInc({mark="none"},Table(x1,y1)),
            PlotInc({mark="none"},Table(x2,y2)),
            {},
            PlotInc({mark="none"},Table(x3,y3)),
            PlotInc({mark="none",style="very thin"},Table(x4,y4))
        )
    DocumentPGFPlots.savefigs(WaveletsEvaluation.DWT.name(w), P)
    removefiles()
    P
end

In [ ]:
plotwavelet(db2)

In [ ]:
plotwavelet(db3)

In [ ]:
plotwavelet(db4)

In [ ]:
plotwavelet(cdf24)

In [ ]:
plotwavelet(cdf35)

In [ ]:
plotwavelet(cdf46)

# Discrete duals

In [ ]:
using InfiniteVectors
using CompactTranslatesDict: signal, CompactPeriodicEquispacedTranslatesDual

In [ ]:
m = 4
dict = 
function plotdiscretedual(dict, m)
    primal_dict = scalingbasis(dict)
    dual_dict = CompactPeriodicEquispacedTranslatesDual(scalingbasis(dict),m)
    s1 = signal(primal_dict, m)
    s2 = signal(dual_dict, m)
    l, r = extrema([support(s1)..., support(s2)...])
    P = @pgf GroupPlot({width=".5\\textwidth",height=".25\\textwidth",group_style = {group_size="1 by 2",},},
            {},
            PlotInc({samples_at=l:r},s1),
            {},
            PlotInc({samples_at=l:r},s2)
        )
    DocumentPGFPlots.savefigs("discrete"*WaveletsEvaluation.DWT.name(wavelet(dict))*string(m), P)
    removefiles()
    P
end

In [ ]:
plotdiscretedual(DaubechiesWaveletBasis(2, 7, Float64, false), m)

In [ ]:
plotdiscretedual(DaubechiesWaveletBasis(3, 7, Float64, false), m)

In [ ]:
plotdiscretedual(DaubechiesWaveletBasis(4, 7, Float64, false), m)

In [ ]:
plotdiscretedual(CDFWaveletBasis(3, 1, 7, typeof(Primal), Float64, false), m)

In [ ]:
plotdiscretedual(CDFWaveletBasis(3, 5, 7, typeof(Primal), Float64, false), m)

In [ ]:
plotdiscretedual(CDFWaveletBasis(4, 2, 7, typeof(Primal), Float64, false), m)

In [ ]:
m = 2; dict=DaubechiesWaveletBasis(2,4)

In [ ]:
using Printf

In [ ]:
itr = rand(10)
myshowvector(stdout,itr,"[", ",", "]", false)

In [ ]:
function myshowvector(io::IO, itr::Union{AbstractArray}, op, delim, cl,
                          delim_one, v=3)
    recur_io = IOContext(io, :SHOWN_SET => itr)
    first = true
    @printf io "\\parbox[t]{.8\\textwidth}{\n"
    print(io, op)
    for i in 1:length(itr)
        x = itr[i]
        show(recur_io, x)
        if i == length(itr)
            delim_one && first && print(io, delim)
            break
        end
        first = false
        print(io, delim)
        print(io, ' ')
        if i != length(itr)
            if rem(i,v)==0 
                @printf io "\\\\ \n"
            end
        end
    end
    print(io, cl)
    @printf io "\n}"
end
function mystring(a) 
    io=IOBuffer()
    myshowvector(io, a, "[", ",", "]", false,3)
    String(take!(io))
end

In [ ]:
function wavelettabular(ws)
    io = IOBuffer()
    @printf io "\\begin{tabular}{|l|c|l|} \n \\hline\n"
    @printf io "type&offset&values \\\\\\hline\n"
    for w in ws
        dict = waveletbasis(w,3)
        primal_dict = scalingbasis(dict)
        dual_dict = CompactPeriodicEquispacedTranslatesDual(scalingbasis(dict),m) 
        s1 = signal(primal_dict, m)
        s2 = signal(dual_dict, m)
        @printf io "\\texttt{%s}&%d&%s\\\\\\hline \n" WaveletsEvaluation.DWT.name(wavelet(dict)) s1.offset mystring(s1.subvec)
        @printf io "&%d&%s\\\\\\hline \n"  s2.offset mystring(s2.subvec)
    end 
    @printf io "\\end{tabular} \n"
    String(take!(io))
end

In [ ]:
open("dualtabular.tikz", "w") do io
   write(io, wavelettabular((db2,db3,db4,cdf31,cdf35,cdf42)))
end;

# Solutions

In [ ]:
using FrameFunWavelets, FrameFunTranslates, FrameFun, DomainSets
using PyPlot

function cplot(c;options...)
    figure(figsize=(5,5))
    P = imshow(abs.(c);cmap="gray_r",norm=matplotlib.colors.LogNorm(eps()),options...)
    P.figaspect=1
    colorbar(shrink=.8)
end
function plotF(F;vmax=1.794, vmin= 0.725, zmax=vmax,zmin=vmin)
    fig = plt.figure(figsize=(5,4))
#     ax = fig.gca()
    ax = fig.gca(projection="3d")
    ax.plot_surface([x[1] for x in g], [x[2] for x in g], F(g);cmap="coolwarm",
                           vmax=vmax, vmin= vmin, edgecolors="k",linewidth=.5,alpha=1.)
    ax.axes.grid(false)
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.set_zlim(zmin, zmax)
    fig
end

In [ ]:
using StaticArrays
D = (.3disk() + SVector(.5,.5)) \ (.1disk() + SVector(.4,.4))

In [ ]:
P = ExtensionFramePlatform(NdDaubechiesPlatform(2,4),D)
P = ExtensionFramePlatform(NdCDFPlatform(2,3,1),D)
f = (x,y)->exp(x*y)
N = (6,6)
L = 4 .* (1 .<< N)
F, A,b,c = FrameFunWavelets.CompactAZ.levelweighed_approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
    REG=pQR_solver,solverstyle=ReducedAZStyle())
@show norm(A*c-b)
FAZS, AAZS,bAZS,cAZS = FrameFunWavelets.CompactAZ.levelweighed_approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
    REG=pQR_solver,solverstyle=SparseAZStyle())
@show norm(AAZS*cAZS-bAZS)
F_ref, A_ref,b_ref,c_ref = approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
    REG=pQR_solver,solverstyle=ReducedAZStyle())
@show norm(A_ref*c_ref-b_ref)
F_refAZS, A_refAZS,b_refAZS,c_refAZS = approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
    REG=pQR_solver,solverstyle=SparseAZStyle())
@show norm(A_refAZS*c_refAZS-b_refAZS)
# F_refDS, A_refDS,b_refDS,c_refDS = approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
#     directsolver=SPQR_solver,solverstyle=DirectStyle())
# @show norm(A_refDS*c_refDS-b_refDS)
F_refD, A_refD,b_refD,c_refD = approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
    directsolver=QR_solver,solverstyle=DirectStyle())
@show norm(A_refD*c_refD-b_refD);

In [ ]:
g =plotgrid(dictionary(F),100)
@show norm(c)
@show norm(cAZS)
@show norm(c_ref)
@show norm(c_refAZS)
# @show norm(c_refDS)
@show norm(c_refD);

In [ ]:
# copts = (cmap="gist_rainbow_r",)
copts = (cmap="gray_r",vmin=1e-10,vmax=10)

In [ ]:
cplot(c;copts...)
PyPlot.savefig("2Dcoefs_AZWR";bbox_inches="tight")

In [ ]:
cplot(c_refD;copts...)
PyPlot.savefig("2Dcoefs_D";bbox_inches="tight")

In [ ]:
cplot(c_ref;copts...)
PyPlot.savefig("2Dcoefs_AZR";bbox_inches="tight")

In [ ]:
g = plotgrid(dictionary(F),512);

In [ ]:
plotF(F;zmin=0)
PyPlot.savefig("f_AZWR";bbox_inches="tight")

In [ ]:
plotF(F_ref;zmin=0)
PyPlot.savefig("f_AZR";bbox_inches="tight")

In [ ]:
plotF(F_refD;zmin=0)
PyPlot.savefig("f_D";bbox_inches="tight")

In [ ]:
P = ExtensionFramePlatform(NdCDBSplinePlatform((3,3)),D)
f = (x,y)->exp(x*y)
N = (6,6)

L = 4 .* (1 .<< N)
N = 1 .<< N

F1_ref, A1_ref,b1_ref,c1_ref = approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
    REG=pQR_solver,solverstyle=ReducedAZStyle())
@show norm(A1_ref*c1_ref-b1_ref)
F1_refD, A1_refD,b1_refD,c1_refD = approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
    directsolver=QR_solver,solverstyle=DirectStyle())
@show norm(A1_refD*c1_refD-b1_refD);
F1_refAZS, A1_refAZS,b1_refAZS,c1_refAZS = approximate(f,P,N;L=L,verbose=false,threshold=1e-10,
    REG=pQR_solver,solverstyle=SparseAZStyle())
@show norm(A1_refAZS*c1_refAZS-b1_refAZS);

In [ ]:
@show norm(c1_ref)
@show norm(c1_refD)
@show norm(c1_refAZS);

In [ ]:
# cplot(c1_ref;copts...)
# PyPlot.savefig("2Dspline_coefs_AZR";bbox_inches="tight")

In [ ]:
plotF(F1_ref;zmin=0)
PyPlot.savefig("fspline_AZR";bbox_inches="tight")

In [ ]:
plotF(F1_refD;zmin=0)
PyPlot.savefig("fspline_D";bbox_inches="tight")

In [ ]:
using Printf
function weightedtabular()
io = IOBuffer() 
@printf io "\\begin{tabular}{l|l|l|l|l|}\n"
@printf io "&\\multicolumn{2}{l}{wavelet extension}&\\multicolumn{2}{|l|}{spline extension}\\\\\n"
@printf io "&\$\\ell^2\$ coef. norm &residual error&\$\\ell^2\$ coef. norm & residual error\\\\\\hline\n"
@printf io "Reduced AZ&%1.2f&%1.2e&%1.2f&%1.2e\\\\\n" norm(c_ref) norm(A_ref*c_ref-b_ref) norm(c1_ref) norm(A1_ref*c1_ref-b1_ref)
@printf io "Weighted reduced AZ&%1.2f&%1.2e&&\\\\\n" norm(c) norm(A*c-b)
@printf io "Pivoted QR&%1.2f&%1.2e&%1.2f&%1.2e\\\\\\hline\n" norm(c_refD) norm(A_refD*c_refD-b_refD) norm(c1_refD) norm(A1_refD*c1_refD-b1_refD) 
@printf io "Sparse AZ&%1.2f&%1.2e&%1.2f&%1.2e\\\\\n" norm(c_refAZS) norm(A_refAZS*c_refAZS-b_refAZS) norm(c1_refAZS) norm(A1_refAZS*c1_refAZS-b1_refAZS)
@printf io "Weighted sparse AZ&%1.2f&%1.2e&&\n" norm(cAZS) norm(AAZS*cAZS-bAZS) 
@printf io "\\end{tabular}\n"
String(take!(io))
end

In [ ]:
open("weightedtabular.tikz", "w") do io
   write(io, weightedtabular())
end;